In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/sol/train_1.csv', index_col=0)
test = pd.read_csv('../../../../data/hi/sol/test_1.csv', index_col=0)

train

,smiles,value
2089,O=C(CN1C(=O)CCC1=O)Nc1ccc(F)c(F)c1F,False
2099,COc1ccc(-c2nnc(NC(=O)c3ccc4ccccc4c3)o2)cc1OC,True
61,O=C(Nc1ccc2ccccc2n1)c1ccc(N2CCOC2=O)cc1,True
89,Cc1c[nH]c(=O)n1-c1ccc(C(=O)Nc2ccc3ccccc3n2)cc1,True
97,O=C(C1CCC(O)CC1)N1CCCN(c2ccc(F)cc2)CC1,False
...,...,...
2143,N#CCCn1nc(C(F)(F)F)cc1O,False
2147,Cc1ccccc1N1C(=O)c2ccc(C(=O)NC(C)(C)C)cc2C1=O,False
2160,CC(C)(C)C(=O)Nc1sc2c(c1C#N)CCC2,True
2165,C[C@@H](c1ccc(F)cc1)n1nnc2cnc3ccc(-c4ccc5ocnc5...,True


# Hyperparameter Optimization

In [4]:
def run_svc_gridsearch(train_fps, test_fps):
    split_index = [-1] * len(train_fps) + [0] * len(test_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + test_fps
    y = train['value'].to_list() + test['value'].to_list()

    params = {
    'C': [0.1, 0.5, 1.0, 2.0, 5.0],
    }
    svc = SVC()

    grid_search = GridSearchCV(svc, params, cv=pds, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    print(best_params)
    svc = SVC(**best_params)
    svc.fit(train_fps, train['value'])

    test_preds = svc.predict(test_fps)
    test_metrics = get_hi_metrics(test, test_preds)
    return test_metrics


In [5]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

test_metrics = run_svc_gridsearch(train_maccs_fps, test_maccs_fps)
print(test_metrics)

Fitting 1 folds for each of 5 candidates, totalling 5 fits
[CV 1/1] END .............................C=0.1;, score=0.457 total time=   0.5s
[CV 1/1] END .............................C=0.5;, score=0.457 total time=   0.4s
[CV 1/1] END .............................C=1.0;, score=0.455 total time=   0.4s
[CV 1/1] END .............................C=2.0;, score=0.476 total time=   0.4s
[CV 1/1] END .............................C=5.0;, score=0.473 total time=   0.4s
{'C': 2.0}
{'roc_auc': 0.595774220383569, 'bedroc': 0.5422338669289191, 'prc_auc': 0.3098643862035937}


# Final Evaluation

In [6]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

    svc = SVC(
        C=2.0
    )
    svc.fit(train_maccs_fps, train['value'])

    train_result = train.copy()
    train_result['preds'] = svc.predict(train_maccs_fps)

    test_result = test.copy()
    test_result['preds'] = svc.predict(test_maccs_fps)

    return train_result, test_result


In [7]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/hi/sol/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/hi/sol/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/hi/sol/svc_maccs/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/hi/sol/svc_maccs/test_{i}.csv')
